In [7]:
import os
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

font_dir = r'C:\Windows\Fonts'
font_list = [os.path.join(font_dir, f) for f in os.listdir(font_dir) if f.endswith('.ttf')]
# font_list = font_list[:30]
usable_fonts=[]
test_text = "1234567890"  # フォントが表示できるかテストするテキスト

image_width = 4  # 画像の幅（インチ）
image_height = 3  # 画像の高さ（インチ）

for idx, font_path in enumerate(font_list):
    try:
        prop = fm.FontProperties(fname=font_path)
        _ = prop.get_family()
        _ = prop.get_name()
        _ = prop.get_style()
        _ = prop.get_variant()
        _ = prop.get_weight()
        _ = prop.get_stretch()
        _ = prop.get_size_in_points()
        _ = prop.get_file()
        if idx+1 not in [56, 68, 86,87,88,113,130,131]:
            usable_fonts.append(font_path)
        # プロット
        # plt.figure(figsize=(image_width, image_height))
        # plt.text(0.1, 0.5, test_text, fontproperties=prop, fontsize=12)
        # plt.title(f"Font {idx + 1}: {os.path.basename(font_path)}")
        # plt.axis('off')
        # plt.show()
    except Exception as e:
        print(f"Not usable: {font_path} - {e}")


In [10]:
from PIL import Image, ImageDraw, ImageFont
import random
import numpy as np
import matplotlib.font_manager as fm
import os
import cv2

np.random.seed(1)
random.seed(1)
# font_dir = r'C:\Windows\Fonts'
# font_list = [os.path.basename(f) for f in os.listdir(font_dir) if f.endswith('.ttf')]
font_list = usable_fonts
# font_list= font_list[:30]
# 画像のサイズを設定する
pixel = 60
N,M=10,10
K=500
dir_name='numbers_line_v2'
dir_path = f'./training_data/{dir_name}'
if not os.path.exists(dir_path):
    os.makedirs(dir_path)
for id in range(K):
    # 画像を作成する
    image = Image.new('RGB', (pixel*M, pixel*N), color=(255, 255, 255))

    # 数字をランダムに選択する
    numbers = [[random.randint(0, 9) for _ in range(M)] for _ in range(N)]
    # numbers = [[random.randint(1, 9) for _ in range(M)] for _ in range(N)]

    # 数字を描画する
    draw = ImageDraw.Draw(image)
    for i in range(N):
        for j in range(M):
            # フォントをランダムに選択する
            # フォントサイズもランダムに上下させる
            font = ImageFont.truetype(random.choice(font_list), size=40+random.randint(-20, 20))
            
            # 数字を描画する
            number = numbers[i][j]
            text_bbox = draw.textbbox((0, 0), str(number), font=font)
            text_width = text_bbox[2] - text_bbox[0]
            text_height = text_bbox[3] - text_bbox[1]
            x = j*pixel + (pixel - text_width) // 2
            y = i*pixel + (pixel - text_height) // 2
            
            # 中心から上下左右に正規分布でずらす
            mu, sigma = 0, 3
            x += int(np.random.normal(mu, sigma))
            y += int(np.random.normal(mu, sigma))
            
            if number:
                draw.text((x, y), f"{number}", font=font, fill=(0, 0, 0))
    #　線を引く
    H,W=np.array(image).shape[:2]
    sigma_l=1
    for num,i in enumerate(range(0, H+1, H//N)):
        if num%3==0:    
            draw.line((i+np.random.normal(0,sigma_l), 0, i+np.random.normal(0,sigma_l), H), fill=(0, 0, 0),width=3)
        else:
            draw.line((i+np.random.normal(0,sigma_l), 0, i+np.random.normal(0,sigma_l), H), fill=(0, 0, 0),width=1)
    for num,j in enumerate(range(0, W+1, W//M)):
        if num%3==0:
            draw.line((0, j+np.random.normal(0,sigma_l), W, j+np.random.normal(0,sigma_l)), fill=(0, 0, 0),width=3)
        else:
            draw.line((0, j+np.random.normal(0,sigma_l), W, j+np.random.normal(0,sigma_l)), fill=(0, 0, 0),width=1)
    # 微小に回転させる
    image=cv2.bitwise_not(np.array(image))
    angle=random.uniform(-1,1)
    image=cv2.cvtColor(np.array(image),cv2.COLOR_RGB2BGR)
    H,W,_=image.shape
    center=(0,0)
    Mat=cv2.getRotationMatrix2D(center,angle,1.0)
    image=cv2.warpAffine(image,Mat,(H,W))
    image=cv2.bitwise_not(image)
    image=Image.fromarray(image)
    # 画像を保存する
    image.save(f'./training_data/{dir_name}/numbers_{id}.png')
    # numberを保存する
    np.save(f'./training_data/{dir_name}/numbers_{id}.npy', numbers)